### Basic RAG retriever using Langchain and Anthropic model

In [2]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

- Estructura basica

In [3]:
from langchain.prompts import PromptTemplate
prompt= PromptTemplate(
    template=""" You are an assistant for questions answering tasks
    You are an incredible (the best of the world) SQL specialist.
    You work is to provide accurate information from the tables presented to you.
    the database that you are going to work with is SQLite, so make sure that when you look for strings only use single quote to refer to them.
    Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use date('now') function to get the current date, if the question involves "today".
    Use the following format:

    Question: Question here
    SQLQuery: SQL Query to run
    SQLResult: Result of the SQLQuery
    Answer: Final answer here
    """,
    input_variables= ['input'],
    )
llm_2=ChatOllama(model='llama3.1',temperature=0)

rag_chain= llm_2 | prompt | StrOutputParser()

In [4]:
db = SQLDatabase.from_uri("sqlite:////Users/main/Desktop/database/llm.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm_2)

In [5]:
tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10385c490>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10385c490>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10385c490>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

# System Prompt
We will also want to create a system prompt for our agent. This will consist of instructions for how to behave.

In [6]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [7]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [8]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': "To answer this question, we need to analyze the customer spending data and group it by country. Let's start by examining the available tables in the database and then query the relevant schema.\n\nFirst, let's list the tables in the database:", 'type': 'text'}, {'id': 'toolu_01Cmv8x6KbPtJk9nh3bbiRZt', 'input': {'tool_input': ''}, 'name': 'sql_db_list_tables', 'type': 'tool_use'}], response_metadata={'id': 'msg_01Angfo6Y7Hq9nW3MfFCCt76', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 1010, 'output_tokens': 107}}, id='run-c9685d0c-273a-4ca2-8b3e-d66f24968fd9-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'toolu_01Cmv8x6KbPtJk9nh3bbiRZt', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1010, 'output_tokens': 107, 'total_tokens': 1117})]}}
----
{'tools': {'messages': [ToolMessage(content='restaurants, restaurants_madrid', name='sq

In [10]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the restaurants_madrid table, how can you improve it?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': "Certainly! To answer your question about the restaurants_madrid table and provide suggestions for improvement, I'll need to examine the table structure and its contents. Let's start by looking at the available tables in the database and then inspect the schema of the restaurants_madrid table.\n\nFirst, I'll list the tables in the database:", 'type': 'text'}, {'id': 'toolu_01EYisLSQjCpBkuymMxB77da', 'input': {'tool_input': ''}, 'name': 'sql_db_list_tables', 'type': 'tool_use'}], response_metadata={'id': 'msg_01AciYwRALcvCb2PgFu8m3kF', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 1016, 'output_tokens': 124}}, id='run-8affd193-7d8f-41ad-8d9b-b8b89fbaf2bb-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'toolu_01EYisLSQjCpBkuymMxB77da', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1016, 'output_tokens': 124, 'total_tokens': 114

In [13]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT name FROM restaurants_madrid")
# albums = query_as_list(db, "SELECT Title FROM Album")
artists[:5]

['',
 'Cantina Canalla Antracita - Restaurante Mexicano',
 'Restaurante FENIX',
 'Yin He',
 'Asador Delicasa']

In [14]:
from langchain.agents.agent_toolkits import create_retriever_tool


vector_db = Chroma.from_texts(artists,embedding=OllamaEmbeddings(model='nomic-embed-text'))
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)



In [25]:
print(retriever_tool.invoke("coke"))

ZAGAL  Pollo&Carbón

Pollo Y Carbón

Pollos al Carbón el Chacarito

Entrecarbón

Pollos Sukao al Carbón


In [29]:
tools = toolkit.get_tools()


In [30]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, state_modifier=system_message)

In [31]:
for s in agent.stream(
    {"messages": [HumanMessage(content="what rating cookalazka has?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': 'To answer this question, we need to find the rating for a restaurant named "cookalazka". However, we should first verify the exact spelling of the restaurant name using the "search_proper_nouns" tool, and then query the database for its rating. Let\'s proceed step by step.\n\nStep 1: Search for the proper noun "cookalazka"', 'type': 'text'}, {'id': 'toolu_01CvbBgD1FsUC6MS9qH2AQz7', 'input': {'query': 'cookalazka'}, 'name': 'search_proper_nouns', 'type': 'tool_use'}], response_metadata={'id': 'msg_016QpyQ8yUdRHHUGYSovvfo4', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 1135, 'output_tokens': 138}}, id='run-33537d6a-e8b2-4806-9a1c-4195d692fbaa-0', tool_calls=[{'name': 'search_proper_nouns', 'args': {'query': 'cookalazka'}, 'id': 'toolu_01CvbBgD1FsUC6MS9qH2AQz7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1135, 'output_tokens': 138, 'total_tokens': 1273})]}